In [1]:
import numpy as np
import scipy.stats as sst
import pandas as pd
from scipy.linalg import solve_banded, solve, solveh_banded
from scipy.interpolate import interp1d
import matplotlib.pyplot as plt 

In [2]:
def bsprice(s, k, r, q, t, sigma, optionType):
    d1 = (np.log(s/k) + (r - q + 0.5*sigma**2)*t) / (sigma*np.sqrt(t))
    d2 = d1 - sigma*np.sqrt(t)
    callOrPut = 1 if optionType.lower()=='call' else -1
    nd1 = sst.norm.cdf(callOrPut*d1)
    nd2 = sst.norm.cdf(callOrPut*d2)
    price = callOrPut*(s*np.exp(-q*t)*nd1 - k*np.exp(-r*t)*nd2)
    return price

In [3]:
s0 = 100
k = 100
r = 0.03
q = 0.01
vol = 0.2
t = 0.25
optionType = 1
maxS = 3*s0
N = 1000
M = 10000
callOrPut = 'call'

In [ ]:
# This codes referenced by prof.Hwang
 
ds = maxS/N # State grid size
dt = t / M # time grid size

i = np.arange(N+1) # index
s = i * ds # state values
a = dt * (vol*s[1:-1])**2 / (2*ds**2)
b = dt * (r-q)*s[1:-1] / (2*ds)

d,m,u = a-b, -2*a - dt * r, a+b

v = np.maximum(optionType * (s-k),0) # payoff

for j in range(M-1, -1, -1): # from M-1 to 0 by -1
    # time t-1 values are directly calculated in explict framework
    temp = d*v[:-2] + (1+m) * v[1:-1] + u * v[2:] 
    # boundary conditions
    v[0] = np.maximum(optionType *(0-k*np.exp(-r*(M-j)*dt)),0) 
    v[N] = np.maximum(optionType*(maxS - k * np.exp(-r * (M - j) * dt)), 0)
    v[1:-1] = temp
f = interp1d(s,v)



In [5]:
f(s0)

array(4.22198926)

In [7]:
def exfdm_vanilla_option(s0, k, r, q, t, vol, optionType, maxS, N, M):
    ds = maxS / N
    dt = t / M
    callOrPut = 1 if optionType.lower()=='call' else -1

    i = np.arange(N+1)
    s = i * ds
    a = dt*(vol*s[1:-1])**2 / (2*ds**2)
    b = dt*(r-q)*s[1:-1] / (2*ds)
    d, m, u = a-b, -2*a-dt*r, a+b

    v = np.maximum(callOrPut*(s-k), 0)

    for j in range(M-1,-1,-1):
        temp = d * v[:-2] + (1 + m) * v[1:-1] + u * v[2:]
        v[0] = np.maximum(callOrPut*(0 - k * np.exp(-r * (M - j) * dt)), 0)
        v[N] = np.maximum(callOrPut*(maxS - k * np.exp(-r * (M - j) * dt)), 0)
        v[1:-1] = temp
    f = interp1d(s,v)
    return pd.DataFrame({"S":s,"V":v}), f(s0)

In [8]:
exfdm_vanilla_option(s0, k, r, q, t, vol, callOrPut, maxS, N, M)

(          S              V
 0       0.0   0.000000e+00
 1       0.3  1.196142e-164
 2       0.6  3.094133e-160
 3       0.9  2.384204e-156
 4       1.2  8.687061e-153
 ...     ...            ...
 996   298.8   1.995034e+02
 997   299.1   1.998141e+02
 998   299.4   2.001250e+02
 999   299.7   2.004360e+02
 1000  300.0   2.007472e+02
 
 [1001 rows x 2 columns],
 array(4.22198926))